In [1]:
import torch
from torch.utils.data import DataLoader
import sys
import numpy as np
sys.path.append('/home3/ebrahim/isr/isr_model_review/BP06/')
from utils import cosine_sim
from datasets import OneHotLetters, OneHotLetters_test
from run_test_trials_EM import run_test_trials_EM
from simulation_one_EM import simulation_one_EM
from RNNcell import RNN_one_layer_EM
import wandb
device = torch.device("cpu")
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels
import pandas as pd
from statsmodels.stats.anova import AnovaRM
import scipy.stats as stats
from scipy.stats import ttest_rel
import pickle
%matplotlib inline

In [2]:
class test_BP06_EM():

    def __init__(self, base):
        
        self.base = base

    def load_model(self, wandbPath, modelPath):

        api = wandb.Api()
        run = api.run(wandbPath)
        self.config = run.config

        # modify paths
        self.config['test_path_lists'] = self.base + self.config['test_path_lists']
        self.config['test_path_set'] = self.base + self.config['test_path_set']
        self.config['test_path_protrusions'] = self.base + self.config['test_path_protrusions']

        saved_info = torch.load(self.base + 'saved_models/' +  modelPath  + '/model_human.pth')
        
        self.model = RNN_one_layer_EM(self.config['input_size'], self.config['hs'], self.config['output_size'], 
                                      noise_std=self.config['noise_std'], feedback_bool=True, bias=False, 
                                      alpha_s=self.config['alpha_s'], storage_capacity=self.config['storage_capacity'])
        
        self.model.load_state_dict(saved_info['model_state_dict'])
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config['lr'])
        self.optimizer.load_state_dict(saved_info['optimizer_state_dict'])
        
    def run_model(self, ll):

        rtt = run_test_trials_EM(self.model, self.optimizer, 0.5)
        rtt.run_model(device, ll, self.config, base='')
        self.h = rtt.h_stacked.numpy()
        self.y_recall_probs = rtt.y_hat_all
        self.y_recall = rtt.y_hat_recall
        self.cache = rtt.cache
        
    def run_simulation_one(self, ll):

        self.sim_one = simulation_one_EM(self.model, self.optimizer, \
                                         self.config['max_length'], self.config['h0_init_val'], \
                                         self.config['test_list_length'])

        self.sim_one.run_model(device, ll, self.config)
        
        self.hidden_activity = self.sim_one.h_stacked
        self.y_recall_probs = self.sim_one.y_hat_all


In [3]:
base = '/home3/ebrahim/isr/isr_model_review/BP06/'
modelPATH_arr = ['EM_opt_Adam_trial_run_dt_True']
wandB_arr = ["ebrahimfeghhi/isr_model_review-BP06/i9tj6scg"]
BP06_mech = test_BP06_EM(base)
BP06_mech.load_model(wandB_arr[0], modelPATH_arr[0])
BP06_mech.run_model(6)

In [8]:
np.asarray(BP06_mech.cache['EM_gate']).ravel()[-1]

<ipython-input-8-d2a0f3924f96>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(BP06_mech.cache['EM_gate']).ravel()[-1]


<generator object run_test_trials_EM.run_model.<locals>.<genexpr> at 0x7ff4faf86900>

In [ ]:
BP06_mech.sim_one.figure_seven_plot(wandb, savePlot=False, displayMode=True)

In [ ]:
def a():
    return 5, 3

In [12]:
a = {'hi': 5, 'bye':10}
for item in a.keys():
    print(item)

hi
bye
